In [3]:
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [12]:
src = cv2.imread("./day_img/day_example4.jpg")
dst = src.copy()
blur = cv2.GaussianBlur(dst, (1, 3), 0)
canny = cv2.Canny(blur, 10, 100)
lines = cv2.HoughLines(canny, 0.8, np.pi, 150, srn = 100, stn = 200, min_theta = 10, max_theta = 20)

for i in lines:
    rho, theta = i[0][0], i[0][1]
    a, b = np.cos(theta), np.sin(theta)
    x0, y0 = a*rho, b*rho

    scale = src.shape[0] + src.shape[1]

    x1 = int(x0 + scale * -b)
    y1 = int(y0 + scale * a)
    x2 = int(x0 - scale * -b)
    y2 = int(y0 - scale * a)

    cv2.line(dst, (x1, y1), (x2, y2), (0, 0, 255), 2)
    cv2.circle(dst, (x0, y0), 3, (255, 0, 0), 5, cv2.FILLED)

cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

C:\Users\YeChan\AppData\Local\Temp/ipykernel_22296/2931281074.py:20: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  cv2.circle(dst, (x0, y0), 3, (255, 0, 0), 5, cv2.FILLED)
